In [24]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.config import config


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from rioplatense_hs.config import config

config

In [26]:
import getpass

# Si no tenemos la configuración, usar esto

#API_KEY = getpass.getpass("Enter your API key: ")

API_KEY = config["OPENAI"]["API_KEY"]

In [27]:
from openai import OpenAI

client = OpenAI(
    api_key=API_KEY
)

In [28]:
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")

In [29]:
dataset["train"][0]

{'id': 343726,
 'title': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad',
 'text': '@usuario Uno menos',
 'context_tweet': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad https://t.co/UrJGRAdEvD',
 'HATEFUL': 1,
 'body': 'Un hombre de 46 años, que cumplía una condena en prisión y fue puesto en libertad por el coronavirus, murió el mismo día que dejó la cárcel baleado por un primo, durante los "festejos" por dejar la prisión.\n\nSari Salem Wardat, de 46 años, recibió un disparo desde atrás en un aparente accidente mientras caminaba hacia miembros de la familia y vecinos que se habían reunido para darle la bienvenida a su casa, en Jordania.\n\nUn video que circuló en las redes sociales \u200bmuestra a Sari llegando a su casa en un auto azul rodeado de tiros de "celebración". El primo se acerca de inmediato a Sari, que sostiene una pistola y la apunta hacia el cielo.\n\nEl detenido

In [30]:

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()

labels = df_test.columns[7:]
df_train["num_hatred"] = df_train[labels].sum(axis=1)

df_test["num_hatred"] = df_test[labels].sum(axis=1)

In [31]:

df_train[df_train["num_hatred"] > 1].sample(2)

def get_text_label(row):
    ret = ""
    for label in labels:
        if row[label] == 1:
            if ret == "":
                ret = label
            else:
                ret = ret + ", " + label
    if ret == "":
        ret = "nada"
    return ret

df_train["text_label"] = df_train.apply(get_text_label, axis=1)
df_test["text_label"] = df_test.apply(get_text_label, axis=1)

In [32]:
import pandas as pd

pd.options.display.max_colwidth = 1000
df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

,title,text,text_label
26846,La impactante historia de la tenista trans que hoy es la N 3 de Argentina en la categoría senior femenino,@usuario Masculino. Masculinooooo.,LGBTI
22391,"Historia: Franco, un hombre trans, tendrá un hijo con Hannah, una mujer trans",@usuario Esto no es normal....\nEn la vida somos hombres y mujeres ..\nAsi es la ciencia...\nLo demaas es puro cuento...,LGBTI
29655,"Jaime Bayly, duro contra Alberto Fernández: ""Es lamentable que usted sea un charlatán""",@usuario Con esa cara de gordo traga tibia opina?,"LGBTI, APPEARANCE"
24225,El meteorólogo Matías Bertolotti se separó a ocho meses de haberse casado,@usuario Porque no la termina con estos putos de mierda,LGBTI
4427,"La historia de Danny Wakefield: ""Soy trans masculino y acabo de dar a luz a mi primer hijo""","@usuario Creete un panda si querés,hasta que no te operes y te pongas un penw sos una mujer disfrazada de hombre o sea un travesti.",LGBTI
13015,"Historia: Franco, un hombre trans, tendrá un hijo con Hannah, una mujer trans",@usuario Y le va a dar la teta un señor? Que quilombo en la cabeza pobre criatura 😒,LGBTI
15306,"El proyecto fotográfico de jóvenes rusos en ""relaciones sexuales no tradicionales"" que desafía al Gobierno de Putin",@usuario Putin debe estar ocupado donde poner los misiles con cabeza nuclear para que caigan en EEUU mira si se va calentar por 2 trolos y 1 una torta directo a Siberia y listo.,LGBTI
20046,"Coronavirus. Bolsonaro, polémico: ""Tenemos que dejar de ser un país de maricas""",@usuario acá en la Arg. Te imaginas ? LOS NUEVOS IFE 18/28 son maricas o VAGOS ?,"LGBTI, CLASS"
28082,Luis Novaresio le dedicó un romántico mensaje a Braulio Bauab por su cumpleaños,@usuario Patéticos!,LGBTI
17863,"El proyecto fotográfico de jóvenes rusos en ""relaciones sexuales no tradicionales"" que desafía al Gobierno de Putin","@usuario Que locura las rusas. Las tortas de acá tienen rastas en la concha y hablan con una ""E"" todo el tiempo","WOMEN, LGBTI"


In [33]:
df_train[df_train["num_hatred"] >= 1].sample(2)

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,num_hatred,text_label
6692,370806,Barrios de Pie convoca a una manifestación en el Obelisco para pedir por el impuesto a la riqueza y la Ley Ramona,@usuario Y k tal di puden trsbajo.vagos.siempre perjudicando a los demas.,Barrios de Pie convoca a una manifestación en el Obelisco para pedir por el impuesto a la riqueza y la Ley Ramona https://t.co/AWNaPkGWzr,1,"La organización social Barrios de Pie convocó una movilización en el Obelisco porteño para el viernes, para pedir por el impuesto a la riqueza y la sanción de la Ley Ramona, frenada en el Congreso, que busca ayudar a las cocineras de los comedores sociales del país.\n\nLa reunión fue convocada para las 11 horas del viernes 14 de agosto bajo el slogan impuesto ala riqueza ya, promocionado en redes sociales y difundido a los medios. El reclamo tiene como eje principal ""impulsar un impuesto a los sectores que concentran la renta en el país"".\n\n""Llama la atención que el gobierno, habiendo planteado que hay que comenzar con 'los últimos', no sea más estricto con 'los primeros'. Hay varios empresarios que durante esta cuarentena se han enriquecido notoriamente, y otros tantos que están lejos de decir que han perdido"", pronuncia el comunicado firmado por Silvia Saravia, por la Mesa Nacional del Movimiento Barrios de Pie.\n\nY agrega: ""Nuestro sistema impositivo es regresivo: pagamos más ...",0,0,0,0,1,0,0,0,0,1,CLASS
14462,345449,Coronavirus en Argentina: se levantó la protesta de los presos en la cárcel de Melchor Romero,@usuario Delincuentes bajenlos por la fuerza,"Coronavirus en Argentina: 150 presos protestan en los techos de una cárcel y piden ""libertad"" para ""no morir"" en prisión https://t.co/RcvBqs0bS3",1,"Las autoridades del Servicio Penitenciario Bonaerense (SPB) informaron esta tarde que se ""retomó la calma"" en la Unidad 10 de Melchor Romero, donde este martes, bien temprano, un grupo de presos inició una protesta para reclamar salidas transitorias y otros beneficios en medio de la pandemia por el coronavirus .\n\nHoras antes, entre 10 y 20 detenidos se subieron a los techos del edificio y, a través de videos enviados con celulares, plantearon pedidos a las autoridades penales y judiciales.\n\nA partir de la intervención de representantes de la Defensoría General de La Plata, la Comisión Provincial por la Memoria (CPM) y directivos del SPB lograron desactivar la manifestación y recuperar el control del edificio que está ubicado en 519 y 176 de esa localidad perteneciente al partido de La Plata.​\n\n""La situación quedó controlada sin que se registraran heridos o lesionados. Los internos preparan un petitorio para el Poder Judicial"", dijeron esta tarde fuentes penitenciarias.\n\nPro...",1,0,0,0,0,0,0,0,1,1,CRIMINAL


In [34]:
# Agregar Chain of thought
# Agregar que es español rioplatense
template_prompt = f"""Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

mujer: Mujeres
lgbti: Género, identidad sexual
racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
clase: Clase, situación socioeconómica o barrio de residencia
discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol, drogas u otros estupefacientes
política: Opinión o ideología política
aspecto: Aspecto o edad
criminal: Antecedentes penales o estar privado de la libertad

En contexto tendremos un titular de una noticia, y en texto el comentario a analizar. Responder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discriminatorio. Pensá paso a paso la respuesta.
##
contexto: Al borde del llanto, Nati Jota pidió disculpas por sus repudiables tuits pero los justificó: 'Quería sumar seguidores'
texto: Show o nada mi amorrr!!
La imbesilidad al palo!
salida: Es un insulto y no menciona nada relacionado a las características protegidas. La respuesta final es "nada"
##
contexto: Violento motín en la cárcel de Devoto: tomaron e incendiaron pabellones
texto: Hermanito lo que son esos grones. Ahí adentro le robaron la billetera al coronavirus
salida: La palabra "grones" significa negros, lo cual alude a su color de piel y aspecto. Además, se refiere a su condición de criminales. La respuesta final es "criminal, racismo, aspecto"
##
contexto: Mayra Mendoza criticó el banderazo contra el Gobierno: "No tiene sentido y es antidemocrático"
texto: Que negrita villera....ni con la plata q roba puede arreglarse
salida: El texto alude a su color de piel, a su condición social, y a su aspecto. La respuesta final es "mujer, racismo, aspecto, clase"
##
contexto: No da tregua: sin ropa y desafiante Flor Peña volvió a mostrar su lado más hot
texto: K lado hot tiene más varices k mi abuela. Tiene mas puesta de espalda k karagian trola
salida: El texto alude a su aspecto y la desprecia por 'trola'. La respuesta final es "mujer, aspecto"
##
contexto: Wuhan celebra el fin de la cuarentena por el coronavirus con un mensaje para el resto del mundo: "Aprendan de nuestros errores"
texto: Chinos culiados de mierda ojalá se mueran todos
salida: Desea que se mueran los chinos por ser los supuestos culpables del COVID-19. La respuesta final es "racismo"
##
contexto: Otro ataque de furia contra un colectivero: una mujer trans le pateó la unidad y le tiró un ladrillazo
texto: @usuario Un tipo operado. Con la fuerza de un hombre y no la de una mujer
salida: El texto alude a que la mujer trans es un hombre. La respuesta final es "lgbti"
##
"""



In [35]:
import re
from pysentimiento.preprocessing import preprocess_tweet

url_regex = r'\burl\b'



def my_preprocess(text):
    text = preprocess_tweet(text, preprocess_hashtags=False, demoji=False)
    text = text.replace("@usuario", "")

    text = re.sub(url_regex, "", text)
    # Replace multiple spaces with one
    text = " ".join(text.split())
    text = text.replace("\n", " ")

    return text

my_preprocess("@usuario Vergonzoso es lo que dice este sr. Los planeros paraguayos, bolivianos, peruanos como cobran el IFE, cobran jubilaciones, votan en Argentina ??? No me diga que NO sabe... es una burla! 👎🤦💩🤡 http://google.com")

'Vergonzoso es lo que dice este sr. Los planeros paraguayos, bolivianos, peruanos como cobran el IFE, cobran jubilaciones, votan en Argentina ??? No me diga que NO sabe... es una burla! 👎🤦💩🤡'

In [36]:
df_test.shape

(11343, 17)

In [37]:
# sample_df = df_test.sample(100)

sample_df = pd.read_csv("../data/sample_df.csv")

In [38]:
sample_df

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,num_hatred,text_label,pred
0,405167,Limpiar la basura arrojada en la marcha por Milagro Sala costó 570 mil pesos,@usuario Aquien le envian la factura ????,Limpiar la basura arrojada en la marcha por Milagro Sala costó 570 mil pesos https://t.co/q9utmSZXKq,0,"Unas cinco mil pesonas protestaron el sábado, convocadas por La Campora y otras agrupaciones kirchneristas, contra la Corte Suprema y por la liberación de Milagro Sala, condenada por el desvío de unos $60 millones para viviendas que no se construyeron y que desaparecieron.\n\nLas imágenes mostraban decenas de bolsas de basura arrojadas en el frente del Palacio de Tribunales. Las cinco mil personas que no guardaron la distancia social se movilizaron y organizaron una puesta en escena a la que denominaron ""El basurero de la democracia"", en alusión de la Corte.\n\nAsí fue que arrojaron unas cuatro toneladas de residuos que, una vez terminada la protesta, fue recogida por el Gobierno de la Ciudad.\n\n¿Cuánto le costó a la Ciudad la limpieza y el arreglo del césped de la Plaza Lavalle? La módica suma de $572 mil que incluyó el hidrolavado y el barrido manual. A eso habría que sumarle el costo por seguridad, es decir, los agentes que fueron sacados de fuerzas especiales para disponerlas ...",0,0,0,0,0,0,0,0,0,0,nada,nada
1,375861,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer: así se inspiró para su segundo film","@usuario Yo no digo que este mal, o que no pase, pero no lo expongan como si fuese un modelo a seguir","Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nhttps://t.co/dt3NtrVJIW",0,"Eldorado, Misiones. ""A treinta brazadas de nado, cruzando el río Paraná, está Paraguay. A 90 kilómetros, la Triple Frontera"", calcula la cineasta, guionista y Majo Staffolani. Habla del lugar donde nació el 13 de junio de 1990, de una familia humilde, un patio de tierra colorada, una calle con el empedrado desparejo y el plato de comida compartido entre los vecinos, por si no alcanzaba a todos. Del escenario, en definitiva, donde la misionera de 30 años disfrazaba sus deseos. ""Los mentía, porque desde los cuatro años se que soy lesbiana"", confiesa quien, por estos días, espera como a un mesías que llegue el 8 de octubre. Será esa la fecha del estreno de Román, su segundo film, que por la pandemia será emitido a través de la plataforma de Cine.ar. Se la escucha tan ansiosa como el torrente de palabras que llegan por el celular.\n\nLa película (que protagonizan Carlo Argento y Gastón Cocchiarale y cuenta con Vanessa Ragone -El secreto de sus ojos- como productora asociada) relata la ...",0,0,0,0,0,0,0,0,0,0,nada,ninguna
2,353018,El ex presidente uruguayo José Mujica renunció al Senado y se retira de la política activa,@usuario se va a cortar pasto con la uña del pie. !!,El ex presidente uruguayo José Mujica renunció al Senado y se retira de la política activa https://t.co/ubQZbsZlhh,0,"El ex presidente uruguayo José Mujica (2010-2015) renunció este martes a su escaño en el Senado y se retiró así definitivamente de la política activa.\n\nLa pandemia de coronavirus precipitó la decisión del ex mandatario de 85 años, que padece una enfermedad inmune. ""Esta situación me obliga, con mucho pesar por mi honda vocación política, a solicitar que gestione mi renuncia a la banca que me otorgó la ciudadanía"", escribió Mujica en una carta leída este martes en sesión extraordinaria del Senado.\n\n""Esto no significa el abandono de la política sino el abandono de la primera fila por entender que un dirigente es el que deja gente que lo supera con ventaja. Me voy agradecido, con muchos recuerdos y honda nostalgia. Me ha echado la pandemia"", sostuvo.\n\nMujica, uno de los principales dirigentes del izquierdista Frente Amplio (FA) y quien llegó a p

In [43]:
outs = {

}

In [44]:
print(template_prompt)

Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

mujer: Mujeres
lgbti: Género, identidad sexual
racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
clase: Clase, situación socioeconómica o barrio de residencia
discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol, drogas u otros estupefacientes
política: Opinión o ideología política
aspecto: Aspecto o edad
criminal: Antecedentes penales o estar privado de la libertad

En contexto tendremos un titular de una noticia, y en texto el comentario a analizar. Responder una o varias de las características separadas por coma, o bien "n

In [45]:
from tqdm import tqdm
import time


def get_completion(prompt):
    return client.chat.completions.create(messages=[{
            "role": "user",
            "content": prompt
    }], model="gpt-3.5-turbo")

for idx, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):

    if idx in outs:
        continue

    contexto = preprocess_tweet(row["context_tweet"], preprocess_hashtags=False, demoji=False, preprocess_handles=False)
    texto = my_preprocess(row["text"])
    prompt = template_prompt + f"contexto: {contexto}\ntexto: {texto}\nsalida: "

    response = get_completion(prompt)

    outs[idx] = response.choices[0].message.content

    time.sleep(0.1)

100%|██████████| 100/100 [02:47<00:00,  1.68s/it]


In [ ]:
import json

with open("outputs.json", "w") as f:
    json.dump(outs, f)


In [ ]:
labels = []

In [ ]:
from collections import Counter

counter = Counter()


for v in outs.values():
    for cl in v.split(","):
        counter[cl.strip().lower()] += 1


In [ ]:

counter

In [46]:
sample_df["pred_cot"] = sample_df.index.map(outs)


In [49]:
sample_df[["context_tweet", "text", "text_label", "pred", "pred_cot"]]

,context_tweet,text,text_label,pred,pred_cot
0,Limpiar la basura arrojada en la marcha por Milagro Sala costó 570 mil pesos https://t.co/q9utmSZXKq,@usuario Aquien le envian la factura ????,nada,nada,"El texto no contiene ninguna expresión discriminatoria. La respuesta final es ""nada"""
1,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nhttps://t.co/dt3NtrVJIW","@usuario Yo no digo que este mal, o que no pase, pero no lo expongan como si fuese un modelo a seguir",nada,ninguna,"El texto no contiene discurso discriminatorio. No se hacen declaraciones de rechazo, enemistad o aborrecimiento hacia un individuo o grupo por poseer una característica protegida. La respuesta final es ""nada""."
2,El ex presidente uruguayo José Mujica renunció al Senado y se retira de la política activa https://t.co/ubQZbsZlhh,@usuario se va a cortar pasto con la uña del pie. !!,nada,nada,"No hay ninguna expresión discriminatoria en el texto. La respuesta final es ""nada""."
3,"""Deje de mentir"": Elisa Carrió criticó a Axel Kicillof y le pidió que se dedique a gobernar https://t.co/pSNvOG8HPI https://t.co/iL7Qb1tR6e",@usuario En Cordoba tenemos que bancar una cuarentena porque los gobernantes de Buenos Aires no saben controlar a la gente. El gobernador solo habla pavadas que son el same reír de toda la argentina,nada,política,"El texto no contiene discurso discriminatorio. La respuesta final es ""nada""."
4,"Hugo Moyano respaldó al Gobierno, criticó a Macri y minimizó las marchas de “mujeres bien vestidas y alimentadas” https://t.co/ZGQQ1DUaZi",@usuario Ladrón korrupto,nada,política,"El mensaje contiene un insulto (""Ladrón korrupto""), pero no hace referencia a ninguna de las características protegidas mencionadas. La respuesta final es ""nada""."
...,...,...,...,...,...
95,Coronavirus: Florencia Kirchner compartió una curiosa reflexión sobre la cuarentena https://t.co/0nI5GKR1NL https://t.co/8KSATPqhFK,@usuario Trabajar nunca no?¿ se saca fotos siempre tirada...🤦‍♀️,nada,ninguna,"El texto alude al hecho de que Florencia Kirchner no trabaja y solo se toma fotos. No contiene discurso discriminatorio. La respuesta final es ""nada""."
96,Ginés González García adelantó que evalúan usar el padrón electoral para la vacunación https://t.co/IajGVulX0M,@usuario Sí. Busquen a sus votantes.,nada,política,"El texto no contiene ningún tipo de discurso discriminatorio. La respuesta final es ""nada""."
97,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nhttps://t.co/dt3NtrVJIW",@usuario @usuario NO DEBERÍA SER NOTICIA... MI COMENTARIO ES :. Y???????????????????........,nada,lgbti,"El texto indica que no debería ser noticia el hecho de que una persona sea lesbiana. No se mencionan características protegidas ni se hace ningún comentario discriminatorio. La respuesta final es ""nada""."
98,"“Fílmame, india horrible”: así se refirió una pareja argentina a vecina de Ciudad de México por la poda de árboles https://t.co/K9sFAt2HKM",@usuario Pero señora usted se vio al espejo para e una mapuche falopera,"RACISM, DISABLED","mujer, racismo, aspecto","El texto utiliza un término despectivo y discriminatorio al referirse a una persona de origen indígena y también menciona su supuesto consumo de drogas. La respuesta final es ""racismo, aspecto""."


In [50]:
sample_df.to_csv("../data/sample_df.csv", index=False)